In [1]:
import pandas as pd

In [2]:
# This data was obtained via a public records request from before I joined the planning commission.
# I've uploaded it here: https://drive.google.com/file/d/1d3DVKjfdyO0GW8t4HYIcqF4RFV45GDou/view?usp=sharing
df_metro = pd.read_excel("data/transit/metro/Spring 2018_Spring 2023_KirklandRoutes.xlsb")
df_metro["group"] = df_metro["group"].replace(
    {
        "A": "prepandemic",
        "B": "postpandemic"
    }
)

In [3]:
df_metro.groupby(["group", "SERVICE_RTE_NUM"]).sum(numeric_only=True)[[
    "AVG_PSNGR_BOARDINGS",
    "AVG_PSNGR_ALIGHTINGS"
]]

AVG_PSNGR_BOARDINGS  AVG_PSNGR_ALIGHTINGS
group        SERVICE_RTE_NUM                                           
postpandemic 245                           2772.5                2785.1
             249                            522.3                 529.7
             250                           2164.9                2168.0
             255                           2567.1                2683.6
             257                            189.3                 191.8
prepandemic  225                            771.0                 798.9
             230                            311.4                 329.2
             231                            285.9                 301.8
             239                            676.5                 676.1
             245                           3684.9                3682.8
             249                            858.5                 857.9
             255                           4741.0                4711.8
             257                            600.0                 628.5

In [4]:
df_route_total = df_metro.groupby(["group", "SERVICE_RTE_NUM"]).sum(numeric_only=True)[[
    "AVG_PSNGR_BOARDINGS",
]].reset_index()

In [5]:
df_route_total

,group,SERVICE_RTE_NUM,AVG_PSNGR_BOARDINGS
0,postpandemic,245,2772.5
1,postpandemic,249,522.3
2,postpandemic,250,2164.9
3,postpandemic,255,2567.1
4,postpandemic,257,189.3
5,prepandemic,225,771.0
6,prepandemic,230,311.4
7,prepandemic,231,285.9
8,prepandemic,239,676.5
9,prepandemic,245,3684.9


In [6]:
df_stops = df_metro.groupby(["group", "SERVICE_RTE_NUM", "STOP_ID"]).sum(numeric_only=True)[[
    "AVG_PSNGR_BOARDINGS"
]].reset_index()

In [7]:
df_stops[df_stops["STOP_ID"] == 74420]

,group,SERVICE_RTE_NUM,STOP_ID,AVG_PSNGR_BOARDINGS
433,postpandemic,255,74420,7.9
1273,prepandemic,255,74420,34.7


In [8]:
df_output = df_stops.pivot_table(
    values="AVG_PSNGR_BOARDINGS",
    index=["SERVICE_RTE_NUM", "STOP_ID"],
    columns="group",
).reset_index().rename(
    {
        "SERVICE_RTE_NUM": "route_number",
        "STOP_ID": "stop_id",
        "prepandemic": "prepandemic_avg_daily_boardings",
        "postpandemic": "postpandemic_avg_daily_boardings",
    },
    axis=1
)
df_output

group,route_number,stop_id,postpandemic_avg_daily_boardings,prepandemic_avg_daily_boardings
0,225,69530,NaN,34.7
1,225,69540,NaN,11.0
2,225,69550,NaN,4.5
3,225,69559,NaN,0.3
4,225,69560,NaN,0.0
...,...,...,...,...
1003,257,85310,0.0,0.5
1004,257,85320,0.0,0.0
1005,257,85330,0.1,0.0
1006,257,85340,0.0,0.4


In [9]:
df_output.to_parquet("data/cleaned_metro_ridership.parquet")